# url2vec

In [1]:
import os
import numpy as np
import pandas as pd
import sys
sys.path.append(os.path.abspath(".."))

#from sklearn import metrics
from hdbscan import HDBSCAN
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
#from gensim.models import Word2Vec
#from url2vec.util.metrics import *
from url2vec.util.plotter import *
from url2vec.model.urlembed import *
from url2vec.util.seqmanager import *

#from sklearn.decomposition import TruncatedSVD
#from sklearn.feature_extraction.text import TfidfVectorizer

### loading files

In [13]:
nocostraint_path = os.getcwd() + "/../dataset/cs.illinois.edu_NoConstraint.words1000.depth10/"

vertex_path      = nocostraint_path + "vertex.txt"
codecontent_map  = get_content_map(vertex_path)

map_path         = nocostraint_path + "urlsMap.txt"
codeurl_map      = get_urlmap(map_path)

sequences_path   = nocostraint_path + "sequenceIDs.txt"
sequences        = get_sequences(sequences_path)


gt = GroundTruth()
ground_truth = [int(gt.get_groundtruth(codeurl_map[code])) for code in codeurl_map]

url2vec = Url2Vec(codeurl_map)

In [20]:
print [codeurl_map[i] for i in codeurl_map]

['https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/john-deere-scholarship-com/', 'https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/jump-trading-scholars/', 'https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/rockwell-collins-scholarsh/', 'https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/spot-trading-scholarship/', 'https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/illinois-cyber-security-sc/', 'https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/senior-100-honorary/', 'https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/cisco-systems-wcs-undergra/', 'https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/crowe-horwa

In [24]:
seq = [
    ["https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/state-farm-computer-scienc/",
     "https://cs.illinois.edu/news/torrellas-co-leads-workshop-popular-parallel-programming/", 
     "https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/state-farm-computer-scienc/"
    ],
    ["https://cs.illinois.edu/news/torrellas-co-leads-workshop-popular-parallel-programming/", 
     "https://cs.illinois.edu/news/cs-illinois-has-strong-showing-sc13/", 
     "https://cs.illinois.edu/news/professor-josep-torrellas-receives-darpa-perfect-award/"
    ]
]

urls = [
    "https://cs.illinois.edu/news/professor-josep-torrellas-receives-darpa-perfect-award/", 
    "https://cs.illinois.edu/news/cs-illinois-has-strong-showing-sc13/", 
    "https://cs.illinois.edu/news/torrellas-co-leads-workshop-popular-parallel-programming/", 
    "https://cs.illinois.edu/prospective-students/undergraduates/undergraduate-scholarships-and-awards/state-farm-computer-scienc/"
]

contents = ["bene bene bene", "si bene bene", "no ciao ciao", "come come come ciao"]

u = Url2Vec(urls)
u.train(sequences=seq, contents=contents, d_dim_red=2)

[-1, -1, -1, -1]

In [5]:
labels = url2vec.train(sequences=seq, contents=codecontent_map)
print(len(labels))

728


In [ ]:
df = pd.DataFrame(url2vec.test(ground_truth), index=set(ground_truth), columns=set(labels))
df

In [ ]:
listcostraint_path = os.getcwd() + "/../dataset/cs.illinois.edu_NoConstraint.words1000.depth10/"
vertex_path_lc      = listcostraint_path + "vertex.txt"
codecontent_map_lc  = get_content_map(vertex_path_lc)

map_path_lc         = listcostraint_path + "urlsMap.txt"
codeurl_map_lc      = get_urlmap(map_path_lc)

sequences_lc        = listcostraint_path + "sequenceIDs.txt"
seq_lc              = get_sequences(sequences_lc)


gt = GroundTruth()
ground_truth = [int(gt.get_groundtruth(codeurl_map[code])) for code in codeurl_map]

The model has to be initialized with a dictionary that associates to every URL a unique code. This is done because for large dataset it takes a lot less resources.

algorithm = HDBSCAN(min_cluster_size=7)

use_w2v = True

use_tfidf = True

e_s g =0

e_min_count = 1

e_window = 10

e_negative = 5

e_size = 48

sequences_list = None

d_max_d = 0.9

d_max_features = 200000

d_min_df = 0.05

d_dim_red = 50

d_tfidf = True

codecontent_map = None


In [ ]:
seq = get_sequences(sequences)
codecontent_map = get_content_map(vertex_path)

k_memb = u2v.train(algorithm=KMeans(n_clusters=15), sequences_list=seq, codecontent_map=codecontent_map)

In [ ]:
seq2 = get_sequences(sequences)
hd_memb = u2v2.train(sequences_list=seq2, codecontent_map=codecontent_map)

In [ ]:
gt = GroundTruth()
ground_truth = [int(gt.get_groundtruth(codeurl_map[code])) for code in codeurl_map]

confusion_table = u2v.test(ground_truth)
pd.DataFrame(confusion_table, index=set(ground_truth), columns=set(u2v.labels_))

In [ ]:
confusion_table_hdbscan = u2v2.test(ground_truth)
pd.DataFrame(confusion_table_hdbscan, index=set(ground_truth), columns=set(u2v2.labels_))

In [ ]:
# k_memb  
labels_pred_k = map(int, k_memb)
labels_pred_h = hd_memb

In [ ]:
print "Homogeneity:\t\t", metrics.homogeneity_score(ground_truth, labels_pred_k)
print "Completeness:\t\t", metrics.completeness_score(ground_truth, labels_pred_k)
print "V Measure Score:\t", metrics.v_measure_score(ground_truth, labels_pred_k)
print "adjusted rand score:\t", metrics.adjusted_rand_score(ground_truth, labels_pred_k)

In [ ]:
print "Homogeneity:\t\t", metrics.homogeneity_score(ground_truth, labels_pred_h) # best 1
print "Completeness:\t\t", metrics.completeness_score(ground_truth, labels_pred_h)  # best 1
print "V Measure Score:\t", metrics.v_measure_score(ground_truth, labels_pred_h) # best 1
print "adjusted rand score:\t", metrics.adjusted_rand_score(ground_truth, labels_pred_h) # best 1